# Coursera Capstone Project.

#### Installation of the required packages and import of the required modules.

In [13]:
#!pip install opencage --quiet
#!conda install beautifulsoup4 --quiet
#!conda install -c conda-forge folium=0.5.0 --yes -q

import lxml
import folium
import requests
import warnings
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
from opencage.geocoder import OpenCageGeocode

#### Setting the options.

In [14]:
%matplotlib notebook
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',9)
pd.set_option('display.max_columns',None)

In [15]:
cities = pd.DataFrame({'city':['London','Toronto','New York']})
cities

,city
0,London
1,Toronto
2,New York


In [16]:
# The code was removed by Watson Studio for sharing.

In [17]:
def getNearbyVenues(cities,category,radius = 500,limit = 3):  
    venues_list = []
    for city in zip(cities):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,city,category,radius,limit)
        results = requests.get(url).json()['response']['groups'][0]['items']        
        venues_list.append([(city[0],
                             v['venue']['id'],
                             v['venue']['name'],
                             v['venue']['location']['lat'],
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name'])
                            for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['city',
                             'venue_id',
                             'venue_name',
                             'venue_latitude',
                             'venue_longitude',
                             'venue_category']
    return(nearby_venues)

In [18]:
cat = '4bf58dd8d48988d1e0931735' # coffee shop
city_venue = getNearbyVenues(cities = cities['city'],category = cat,limit = 3)    
city_venue

,city,venue_id,venue_name,venue_latitude,venue_longitude,venue_category
0,London,52764ec8498e83fe786a0595,Afternoon Tea at the Corinthia Hotel,51.506572,-0.124350,Tea Room
1,London,4cdadd2dc409b60cac66d11a,Notes Music & Coffee,51.509696,-0.126985,Coffee Shop
2,London,4b228bfef964a520c34824e3,Café In The Crypt,51.508962,-0.126729,Café
3,Toronto,4b896ec4f964a520bd3532e3,Hotel Gelato,43.703478,-79.414311,Café
4,Toronto,4bdc79352a3a0f47a800b3b6,The Mad Bean Coffee House,43.703529,-79.413698,Coffee Shop
5,Toronto,4b0feea1f964a5206a6623e3,Starbucks,43.700598,-79.427433,Coffee Shop
6,New York,5b819467598e64002c8b3db7,Gotan,40.740699,-73.993268,Café
7,New York,50b10a97e4b0c59eea49afef,DAVIDsTEA,40.742090,-73.993378,Tea Room
8,New York,5a5e29ae0868a24e8b9a8f1e,Maman,40.743626,-73.990211,Café


In [19]:
def get_venue_data(venues,limit = 3):
    venues_list = []
    for vn in venues:
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(vn,CLIENT_ID,CLIENT_SECRET,VERSION)
        results = requests.get(url).json()['response']['venue']
        venues_list.append([(results['id'],
                             results['name'],
                             results['location']['formattedAddress'],
                             results['location']['lat'],
                             results['location']['lng'],
                             results['canonicalUrl'],
                             results['categories'][0]['name'],
                             results['verified'],
                             results['stats']['tipCount'],
                             results['price']['tier'],
                             results['price']['message'],
                             results['price']['currency'],
                             #results['hasMenu'],
                             results['likes']['count'],
                             results['likes']['groups'][0]['type'],
                             results['likes']['groups'][0]['count'],
                             results['likes']['summary'],
                             results['rating'],
                             results['ratingSignals'],
                             #results['menu']['url'],
                             results['photos']['count'],
                             #results['reasons']['items'][0]['summary'],
                             results['tips']['count'],
                             results['tips']['groups'][0]['type'],
                             results['tips']['groups'][0]['name'],
                             results['tips']['groups'][0]['count'],
                             results['tips']['groups'][0]['items'],
                             results['hours']['status'],
                             results['hours']['timeframes'][0]['days'],
                             #results['parent']['id']
                            )])

    df_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    df_venues.columns = ['venue_id',
                         'name',
                         'formattedAddress',
                         'lat',
                         'lng',
                         'canonicalUrl',
                         'categories',
                         'verified',
                         'tipCount',
                         'tier',
                         'message',
                         'currency',
                         'count',
                         'groupsType',
                         'groupsCount',
                         'summary',
                         'rating',
                         'ratingSignals',
                         'photos',
                         'tips',
                         'tips-groups-type',
                         'tips-groups-name',
                         'tips-groups-count',
                         'tips-groups-items',
                         'status',
                         'days']
    return(df_venues)

In [21]:
df = pd.concat([city_venue,get_venue_data(city_venue['venue_id'],limit = 3)],axis = 1,join = 'inner')

In [22]:
df

,city,venue_id,venue_name,venue_latitude,venue_longitude,venue_category,venue_id,name,formattedAddress,lat,lng,canonicalUrl,categories,verified,tipCount,tier,message,currency,count,groupsType,groupsCount,summary,rating,ratingSignals,photos,tips,tips-groups-type,tips-groups-name,tips-groups-count,tips-groups-items,status,days
0,London,52764ec8498e83fe786a0595,Afternoon Tea at the Corinthia Hotel,51.506572,-0.124350,Tea Room,52764ec8498e83fe786a0595,Afternoon Tea at the Corinthia Hotel,"[10 Whitehall Pl, London, Greater London, SW1A...",51.506572,-0.124350,https://foursquare.com/v/afternoon-tea-at-the-...,Tea Room,False,12,2,Moderate,£,34,others,34,34 Likes,8.8,45,53,12,others,All tips,12,"[{'id': '5d6614d799438a000c44b901', 'createdAt...",Closed until 1:00 PM,Mon–Sun
1,London,4cdadd2dc409b60cac66d11a,Notes Music & Coffee,51.509696,-0.126985,Coffee Shop,4cdadd2dc409b60cac66d11a,Notes Music & Coffee,"[31 St Martins Ln, London, Greater London, WC2...",51.509696,-0.126985,https://foursquare.com/v/notes-music--coffee/4...,Coffee Shop,False,166,3,Expensive,£,469,others,469,469 Likes,8.3,681,350,166,others,All tips,166,"[{'id': '4d1c8cb46cf9721e54a82667', 'createdAt...",Open until 10:00 PM,Mon–Wed
2,London,4b228bfef964a520c34824e3,Café In The Crypt,51.508962,-0.126729,Café,4b228bfef964a520c34824e3,Café In The Crypt,"[St. Martin-in-the-fields, London, Greater Lon...",51.508962,-0.126729,https://foursquare.com/v/caf%C3%A9-in-the-cryp...,Café,False,65,1,Cheap,£,226,others,226,226 Likes,8.1,330,236,65,others,All tips,65,"[{'id': '4ccca01bc0378cfaba0b8848', 'createdAt...",Closed until 8:00 AM,Mon–Tue
3,Toronto,4b896ec4f964a520bd3532e3,Hotel Gelato,43.703478,-79.414311,Café,4b896ec4f964a520bd3532e3,Hotel Gelato,"[532 Eglinton Ave. W (at Heddington Ave.), Tor...",43.703478,-79.414311,https://foursquare.com/hotelgelato,Café,True,35,1,Cheap,$,53,others,53,53 Likes,8.7,87,62,35,others,All tips,35,"[{'id': '51e01bde498e8952b36469b2', 'createdAt...",Closed until 8:00 AM,"Mon–Fri, Sun"
4,Toronto,4bdc79352a3a0f47a800b3b6,The Mad Bean Coffee House,43.703529,-79.413698,Coffee Shop,4bdc79352a3a0f47a800b3b6,The Mad Bean Coffee House,"[519 Eglinton Avenue West (Heddington Ave), To...",43.703529,-79.413698,https://foursquare.com/v/the-mad-bean-coffee-h...,Coffee Shop,True,24,1,Cheap,$,19,others,19,19 Likes,8.2,40,34,24,others,All tips,24,"[{'id': '4e455275d4c065cb6b8f9367', 'createdAt...",Closed until 7:00 AM,Mon–Fri
5,Toronto,4b0feea1f964a5206a6623e3,Starbucks,43.700598,-79.427433,Coffee Shop,4b0feea1f964a5206a6623e3,Starbucks,"[900 Eglinton Ave W (at Hilltop Rd.), Toronto ...",43.700598,-79.427433,https://foursquare.com/v/starbucks/4b0feea1f96...,Coffee Shop,True,11,2,Moderate,$,35,others,35,35 Likes,8.0,51,36,11,others,All tips,11,"[{'id': '4da9baa8fa8cc76497213a9c', 'createdAt...",Closed until 5:30 AM,Mon–Fri
6,New York,5b819467598e64002c8b3db7,Gotan,40.740699,-73.993268,Café,5b819467598e64002c8b3db7,Gotan,"[37 W 20th St, New York, NY 10011, United States]",40.740699,-73.993268,https://foursquare.com/v/gotan/5b819467598e640...,Café,False,10,1,Cheap,$,49,others,49,49 Likes,8.6,72,20,10,others,All tips,10,"[{'id': '5c97aa7a04d1ae002c6cc55c', 'createdAt...",Closed until 7:00 AM,Mon–Fri
7,New York,50b10a97e4b0c59eea49afef,DAVIDsTEA,40.742090,-73.993378,Tea Room,50b10a97e4b0c59eea49afef,DAVIDsTEA,"[688 Avenue of the Americas (at W 22nd St), Ne...",40.742090,-73.993378,https://foursquare.com/v/davidstea/50b10a97e4b...,Tea Room,True,49,1,Cheap,$,132,others,132,132 Likes,8.5,184,97,49,others,All tips,49,"[{'id': '50ec63c9e4b00e1df3ef9a09', 'createdAt...",Closed until 7:00 AM,Mon–Fri
8,New York,5a5e29ae0868a24e8b9a8f1e,Maman,40.743626,-73.990211,Café,5a5e29ae0868a24e8b9a8f1e,Maman,"[22 W 25th St (btwn 6th Ave & Broadway), New Y...",40.743626,-73.990211,https://foursquare.com/v/maman/5a5e29ae0868a24...,Café,False,33,1,Cheap,$,141,others,141,141 Likes,8.9,201,144,33,others,All tips,33,"[{'id': '5d90f2f0a6e0fa0007a5aeaf', 'createdAt...",Closed until 7:00 AM,Mon–

In [23]:
city = 'New York'
one_city_venues = city_venue.copy()
one_city_venues = one_city_venues.loc[one_city_venues['city'] == city,:]

one_city_venue_map = folium.Map(location = [one_city_venues['venue_latitude'].mean(),one_city_venues['venue_longitude'].mean()],zoom_start = 15)
for venue,lat,lng in zip(one_city_venues['venue_name'],one_city_venues['venue_latitude'],one_city_venues['venue_longitude']):
    label = folium.Popup('{}'.format(venue),parse_html = True)
    folium.CircleMarker([lat,lng],radius = 5,popup = label,color = 'blue',fill = True,fill_color = '#3186cc',fill_opacity = 0.7,
                        parse_html = False).add_to(one_city_venue_map)
one_city_venue_map